In [1]:
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras

%load_ext tensorboard

import numpy as np
import pandas as pd
import os

np.random.seed(42)

# graphic option tuning
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# training DNN in CIFAR10 dataset
###   
* 60000 color images
* 32*32 pixel, 3 channel
* 10 class labels
###  
task : multiclass classification

### 1. data load

In [3]:
# train : test = 50000 : 10000
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

# train : valid = 45000 : 5000
X_train = X_train_full[5000:]
y_train = y_train_full[5000:]
X_valid = X_train_full[:5000]
y_valid = y_train_full[:5000]

### 2. model build

In [5]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential()
# input define
model.add(keras.layers.Flatten(input_shape = [32,32,3]))
# 20-hidden-layers have 100 neurons
# using ELU activation & He initialization
for _ in range(20):
    model.add(keras.layers.Dense(100,
                                 activation = "elu",
                                 kernel_initializer = "he_normal"))
# output layer
model.add(keras.layers.Dense(10, activation = "softmax"))

optimizer = keras.optimizers.Nadam(lr = 5e-5)
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = optimizer,
              metrics = ["accuracy"])

In [7]:
# callbacks define
early_stopping_cb = keras.callbacks.EarlyStopping(patience = 20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("my_cifar10_model.h5",
                                                      save_best_only = True)
run_index = 1
run_logdir = os.path.join(os.curdir, "my_cifar10_logs", "run_{:03d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [8]:
%tensorboard --logdir=./my_cifar10_logs --port=6006

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 10496.

### 3. model fitting

In [9]:
model.fit(X_train, y_train, epochs = 100,
          validation_data = (X_valid, y_valid),
          callbacks = callbacks)

Train on 45000 samples, validate on 5000 samples
Epoch 1/100
45000/45000 [==============================] - 16s 347us/sample - loss: 4.3406 - accuracy: 0.1496 - val_loss: 2.1599 - val_accuracy: 0.2122
Epoch 2/100
45000/45000 [==============================] - 15s 339us/sample - loss: 2.0637 - accuracy: 0.2351 - val_loss: 1.9966 - val_accuracy: 0.2604
Epoch 3/100
45000/45000 [==============================] - 14s 317us/sample - loss: 1.9546 - accuracy: 0.2744 - val_loss: 2.0142 - val_accuracy: 0.2626
Epoch 4/100
45000/45000 [==============================] - 15s 323us/sample - loss: 1.8816 - accuracy: 0.3042 - val_loss: 1.8472 - val_accuracy: 0.3152
Epoch 5/100
45000/45000 [==============================] - 16s 352us/sample - loss: 1.8237 - accuracy: 0.3319 - val_loss: 1.8066 - val_accuracy: 0.3442
Epoch 6/100
45000/45000 [==============================] - 17s 386us/sample - loss: 1.7730 - accuracy: 0.3530 - val_loss: 1.7865 - val_accuracy: 0.3480
Epoch 7/100
45000/45000 [==============

45000/45000 [==============================] - 21s 465us/sample - loss: 1.1457 - accuracy: 0.5904 - val_loss: 1.5596 - val_accuracy: 0.4792
Epoch 55/100
45000/45000 [==============================] - 21s 460us/sample - loss: 1.1378 - accuracy: 0.5930 - val_loss: 1.6062 - val_accuracy: 0.4638
Epoch 56/100
45000/45000 [==============================] - 21s 464us/sample - loss: 1.1306 - accuracy: 0.5945 - val_loss: 1.5691 - val_accuracy: 0.4750


![learning_curve](https://github.com/ChangDaeO730/IMAGE_LINK_reposit/blob/main/execution_result/learning_curve.png)

#### evaluating 

In [16]:
model.evaluate(X_valid, y_valid)

5000/5000 [==============================] - 0s 81us/sample - loss: 1.5691 - accuracy: 0.4750


[1.5690993909835815, 0.475]

In [17]:
# best model for validation set
keras.models.load_model("my_cifar10_model.h5").evaluate(X_valid, y_valid)

model = keras.models.load_model("my_cifar10_model.h5")

5000/5000 [==============================] - 1s 124us/sample - loss: 1.4964 - accuracy: 0.4772
